!pip install -U openmim
!mim install mmengine
!mim install mmcv
!git clone https://github.com/open-mmlab/mmpretrain.git
%cd mmpretrain
!pip install -e . 

In [ ]:
import numpy as np
import mmcv
import torch
from torch import nn
#from torchvision import transforms,datasets
from mmengine import Config
from mmengine.runner import Runner

from mmpretrain import list_models,get_model,ImageClassificationInferencer
#inference_model每次都要初始化模型，不好用，所以直接用任务自带的类执行推理

In [ ]:
img = '/root/VOC2012/JPEGImages/2007_000027.jpg'
#list_models()
model = get_model('vgg11_8xb32_in1k',pretrained=True)#也可以给定权重

In [ ]:
infer = ImageClassificationInferencer(model)
infer(img)[0]

In [ ]:
#可视化特征图
# pip install grad-cam
python tools/visualization/vis_cam.py \
   demo/bird.JPEG 图片\
   configs/vision_transformer/vit-base-p16_64xb64_in1k-384px.py 模型\
   https://download.openmmlab.com/mmclassification/v0/vit/finetune/vit-base-p16_in21k-pre-3rdparty_ft-64xb64_in1k-384_20210928-98e8652b.pth 参数 \
   --vit-like \
   --target-layers 'backbone.layers.11.ln1'#指定一层特征
    --preview-model#查看所有层的名字
     --method GradCAM
   # GradCAM++, XGradCAM, EigenCAM, EigenGradCAM, LayerCAM

In [ ]:
#看学习率的变化情况
python tools/visualization/vis_scheduler.py \
configs/swin_transformer/swin-base_16xb64_in1k.py \
--dataset-size 1281167 --ngpus 16

##### 以参数为主要任务，可以直接修改cfg文件，可以载入后再修改，反正传入的是个类

In [ ]:
cfg = Config.fromfile('/root/mmpretrain/configs/resnext/resnext50-32x4d_8xb32_in1k.py')
cfg.seed = 0
cfg.gpu_ids = [0]#设备指定
cfg.device='cuda'

In [ ]:
#参数
cfg.model.head.num_classes=10
cfg.model

In [ ]:
#方便修改LOADER中的类型
cfg.dataset_type='VOC'
cfg.dataset_type

In [ ]:
#数据预处理
cfg['data_preprocessor'].num_classes=10
cfg['data_preprocessor']

In [ ]:
#给loader用的
cfg.train_pipeline
cfg.test_pipeline

In [ ]:
#训练集不是设置dataset,而是直接设置loader

cfg.train_dataloader.dataset.type='VOC'
cfg.train_dataloader.dataset.data_root='/root/VOC2012/'
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader

In [ ]:
cfg.train_cfg

In [ ]:
#cfg.val_dataloader.dataset.type='VOC'
#cfg.val_dataloader.dataset.data_root='/root/VOC2012/'
#cfg.val_dataloader.dataset.pipeline = cfg.val_dataloader
cfg.val_dataloader=None

In [ ]:
cfg.val_evaluator=None
cfg.val_cfg=None

In [ ]:
cfg.test_dataloader.dataset.type='VOC'
cfg.test_dataloader.dataset.data_root='/root/VOC2012/'
cfg.test_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.test_dataloader

In [ ]:
cfg.optim_wrapper

In [ ]:
#学习率变化
cfg.param_scheduler

In [ ]:
#具体训练形式
cfg.train_cfg
cfg.val_cfg
cfg.test_cfg

In [ ]:
#自动缩放学习率，按batch大小决定
cfg.auto_scale_lr

In [ ]:
cfg.default_scope

In [ ]:
#钩子
cfg.default_hooks

In [ ]:
#唤醒参数
cfg.env_cfg

In [ ]:
cfg.vis_backends
#可以有多个后端
#WandbVisBackend
#TensorboardVisBackend
#MLflowVisBackend
#ClearMLVisBackend
#NeptuneVisBackend
#DVCLiveVisBackend
#AimVisBackend
#LocalVisBackend

In [ ]:
cfg.visualizer
#不同任务有单独的

In [ ]:
cfg.log_level

In [ ]:
#这是从给定的继续
cfg.load_from
#这是继续
cfg.resume

In [ ]:
cfg.randomness

In [ ]:
cfg.seed

In [ ]:
cfg.gpu_ids

In [ ]:
cfg.device

In [ ]:
cfg.work_dir=''

In [ ]:
runner=Runner.from_cfg(cfg)

In [ ]:
runner.train()

In [ ]:
runner.val()

# 1 读取参数文件

In [ ]:

#读取所有的参数,参数文件就在当下文件夹中
cfg = Config.fromfile('./configs/resnext/resnext50_32x4d_b32x8_imagenet.py')
cfg.seed = 0
cfg.gpu_ids = [0]#设备指定m
cfg.device='cuda'

# 2 使用模型

In [ ]:
#模型修改
cfg.model.head.num_classes = 10 
#MIXup
cfg.model['train_cfg']=dict(
        augments=dict(type='BatchMixup', alpha=0.2, num_classes=10,
                      prob=1.))
#Labelsmooth
cfg.model.head.loss = {'type': 'LabelSmoothLoss', 'loss_weight': 1.0,'label_smooth_val':0.1}
 

# 3数据集

In [ ]:
#数据路径
training_data_path='/kaggle/input/10-monkey-species/training/training/'
valid_data_path='/kaggle/input/10-monkey-species/validation/validation/'

In [ ]:
from mmpretrain.registry import DATASETS
from torch.utils.data import Dataset
from mmengine.dataset import Compose
data = load_dataset('biglam/illustrated_ads')
data_split=data['train'].train_test_split(test_size=0.2)
transform =[]
for i in config['train_pipeline']:
    transform.append(TRANSFORMS.build(i))
@DATASETS.register_module()
class TEXTID(Dataset):
    def __init__(self,data,transform):
        super().__init__()
        self.data = data
        self.pipeline = Compose(transform)
    def __getitem__(self,index):
        result = {'img':self.data[index]['image'],'gt_label':self.data[index]['label']}
        out =self.pipeline(result)
        return out
    def __len__(self):
        return len(self.data)
    
    

In [ ]:
cfg.dataset_type = 'CustomDataset'
#现在不再需要构造data，直接构造dataloader了


train_dataloader = Runner.build_dataloader(
            cfg.data.train,
            cfg.data.samples_per_gpu,
            cfg.data.workers_per_gpu,
            # cfg.gpus will be ignored if distributed
            num_gpus=len(cfg.gpu_ids),
            dist=False,
            round_up=True,
            seed=cfg.seed,
            sampler_cfg=None) 
val_dataset = build_dataset(cfg.data.val)

val_dataloader =build_dataloader(
            cfg.val_data,
            cfg.data.samples_per_gpu,
            cfg.data.workers_per_gpu,
            # cfg.gpus will be ignored if distributed
            num_gpus=len(cfg.gpu_ids),
            dist=False,
            round_up=True,
            seed=cfg.seed,
            sampler_cfg=None) 

# 4 优化器封装

In [ ]:
#optimizer = torch.optim.SGD(mymodel.parameters(), lr=0.01, momentum=0.9)
#optim_wrapper=dict(optimizer=optimizer)
#optimizer_2 =  torch.optim.SGD(mymodel.parameters(), lr=0.01, momentum=0.9)
#optim_wrapper_2=dict(optimizer=optimizer_2)
#optim_dict = OptimWrapperDict(gen=optim_wrapper, disc=optim_wrapper_2)

cfg.optimizer['lr']=0.1*cfg.data['samples_per_gpu']/256
#字典形式
optimizer = cfg.optimizer 
optim_wrapper = dict(type='OptimWrapper', optimizer=optimizer)
#混合精度封装
#optim_wrapper = dict(type='AmpOptimWrapper', optimizer=optimizer, accumulative_counts=2)
#分段切割模型优化器
#optimizer = SGD([{'params': model.backbone.parameters()},
 #                 {'params': model.head.parameters(), 'lr': 1e-3}],
#                  lr=0.01,momentum=0.9)

# 5优化器参数调整

In [ ]:
#scheduler = MultiStepLR(optimizer, milestones=[8, 11], gamma=0.1)
#scheduler = dict(type='CosineAnnealingLR', by_epoch=True, T_max=12, convert_to_iter_based=True)

#多阶段参数变化
scheduler = [
    # 线性学习率预热调度器
    dict(type='LinearLR',
         start_factor=0.001,
         by_epoch=False,  # 按迭代更新学习率
         begin=0,
         end=50),  # 预热前 50 次迭代
    # 主学习率调度器
    dict(type='MultiStepLR',
         by_epoch=True,  # 按轮次更新学习率
         milestones=[8, 11],
         gamma=0.1,
         begin=50,
         end=900)
]

# 修改钩子

In [ ]:
default_hooks = dict(
    runtime_info=dict(type='RuntimeInfoHook'),
    timer=dict(type='IterTimerHook'),
    #sampler_seed=dict(type='DistSamplerSeedHook'),
    #logger=dict(type='LoggerHook'),
    param_scheduler=dict(type='ParamSchedulerHook'),
    checkpoint=dict(type='CheckpointHook', interval=2),
    #cfg.checkpoint_config.interval = 12
)

# 评估标准

In [ ]:

#METRICS._module_dict.pop('Accuracy')
@METRICS.register_module()  # 将 Accuracy 类注册到 METRICS 注册器
class Accuracy(BaseMetric):
    default_prefix = 'ACC'  # 设置 default_prefix
    def process(self, data_batch: Sequence[dict], predictions: Sequence[dict]):
        #process(data_samples=outputs, data_batch=data_batch)
        # 取出分类预测结果和类别标签
        result = {
            'pred': predictions,
            'gt': data_batch['gt_label']
        }

        # 将当前 batch 的结果存进 self.results
        self.results.append(result)

    def compute_metrics(self, results: List):
        # 汇总所有样本的分类预测结果和类别标签
        preds = np.concatenate([res['pred'] for res in results])
        gts = np.concatenate([res['gt'] for res in results])
        # 计算分类正确率
        acc = (preds == gts).sum() / preds.size
        # 返回评测指标结果
        return {'accuracy': acc}

In [ ]:
randomness=dict(seed=1, deterministic=True, diff_rank_seed=False)

In [ ]:
train_cfg = dict(
    by_epoch=True,
    max_epochs=100,
    val_begin=1,  # 从第 20 个 epoch 开始验证
    val_interval=2  # 每隔一个 epoch 进行一次验证
)
#没有的话要全部设为None
val_cfg = dict()
val_evaluator = dict(type='Accuracy')

test_cfg = dict()
test_evaluator = dict(type='Accuracy')
# 初始化执行器
runner = Runner(mymodel,
                work_dir='./train_mnist',  # 工作目录，用于保存模型和日志
                train_cfg=train_cfg,       #训练的流程
                train_dataloader=train_dataloader,  
                
                val_dataloader=val_dataloader,
                val_cfg=val_cfg,
                val_evaluator=val_evaluator,
                
                test_dataloader=val_dataloader,
                test_cfg=val_cfg,
                test_evaluator=val_evaluator,  
                
                #randomness=randomness,  # 配置随机种子
                #env_cfg=env_cfg,  # 配置环境变量
                
                optim_wrapper=optim_wrapper,
                param_scheduler=scheduler,
                default_hooks=default_hooks,
               #load_from='./train_mnist/epoch_2.pth',
               #resume=True
               )
# 执行训练
runner.train()

In [ ]:
#测试
#test_cfg = dict()
#test_evaluator = dict(type='Accuracy')
#runner = Runner(model=mymodel, test_dataloader=test_dataloader, test_evaluator=test_evaluator,test_cfg=test_cfg,
#                load_from='./train_mnist/epoch_2.pth', work_dir='./test_mnist')

# 执行测试
runner.test()

# 绘制工具

In [40]:
from mmengine.visualization import Visualizer
import mmcv
import torch
import numpy as np

In [8]:
img = mmcv.imread('E:\dog.jpg',channel_order='rgb')

In [44]:
vis  = Visualizer(image = img)

In [53]:
#vis.draw_texts(texts='the dog is running',positions=torch.tensor([10,20]),)
#vis.draw_polygons(np.array([[12,20],[20,40],[40,100],[100,80]]))
#vis.draw_bboxes(np.array([[10,20,30,40],[80,100,120,140]]))
#vis.draw_lines(np.array([16,39]),np.array([400,500]))
#和原图一样大的mask
#vis.draw_binary_masks(torch.ones(576,768,dtype=torch.bool))
#vis.show()

# 单机多卡

In [ ]:
python -m torch.distributed.launch --nproc_per_node=8 examples/train.py --launcher pytorch

# 部署 onnx

In [ ]:
from mmdeploy.apis import torch2onnx
img = 'demo.jpg'
work_dir = 'work_dir'
save_file = 'fcos.onnx'
deploy_cfg = ('configs/mmdet/detection/detection_onnxruntime_dynamic.py')
model_cfg = ('mmdetection/configs/fcos/fcos_r50_caffe_fpn_gn-head_1x_coco.py')
#model_checkpoint = ('checkpoints/fcos_r50_caffe_fpn_gn-head_1x_coco-821213aa.pth')
#device = 'cpu'
torch2onnx(img, work_dir, save_file, deploy_cfg, model_cfg, 
          # model_checkpoint, device
          )

In [ ]:
import gradio as gr
#输入文本处理程序
def greet(name):
    return "Hello " + name + "!"
#接口创建函数
#fn设置处理函数，inputs设置输入接口组件，outputs设置输出接口组件
#fn,inputs,outputs都是必填函数
demo = gr.Interface(fn=greet, 
                    inputs=["text","image"], 
                    outputs=["text"])
demo.launch()